# BEE2041 Empirical Project Blog

In [59]:
pip install selenium --upgrade

Requirement already up-to-date: selenium in c:\users\socor\anaconda3\lib\site-packages (4.19.0)
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.6.20
    Uninstalling certifi-2020.6.20:
      Successfully uninstalled certifi-2020.6.20
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9
Note: you may need to restart the kernel to use updated packages.


ERROR: requests 2.24.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 2.2.1 which is incompatible.


We need a list of all PCCs/force areas. let us scrape that list:


In [46]:
url = 'https://www.police.uk/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .big-button,body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error

In [58]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome(r'C:\Users\socor\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
driver.get('https://www.police.uk/')

# Let the page load. Consider using WebDriverWait for better practice.
import time
time.sleep(5)  # Adjust sleep time as needed.

soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup)

driver.quit()


SyntaxError: invalid syntax (<ipython-input-58-c29cab7ae0dd>, line 1)

In [60]:
pip install cloudscraper

  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
Note: you may need to restart the kernel to use updated packages.


ERROR: selenium 4.19.0 has requirement urllib3[socks]<3,>=1.26, but you'll have urllib3 1.25.11 which is incompatible.


In [62]:
import cloudscraper
url = 'https://www.police.uk/'
scraper = cloudscraper.create_scraper()
res= scraper.get(url)
print(res.status_code)
res.text

403


'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .big-button,body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error-tex

In [66]:
import cfscrape
url = 'https://www.police.uk/'
scrape = cfscrape.create_scraper()
res = scrape.get(url)
print(res.status_code)    

403


In [64]:
pip install cfscrape

Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time

# Specify the path to chromedriver if it's not in your PATH
chromedriver_path = r"C:\Users\socor\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Initialize the WebDriver (assuming Chrome)
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

try:
    # Navigate to a website
    driver.get("http://police.uk")
    
    # Wait for 5 seconds to see the page
    time.sleep(5)
    
    # Optionally, interact with the website
    # For example, search for 'Selenium' in Wikipedia
    # search_box = driver.find_element_by_name('q')
    # search_box.send_keys('Selenium')
    # search_box.send_keys(Keys.RETURN)
    # time.sleep(5)
    
    print("Selenium is working fine!")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()

Selenium is working fine!


Having established that Selenium is capable of accessing the police.uk website, let's start building an ethical bot! Firstly, we accessed the https://police.uk/robots.txt page and found certain URLs needed to be disallowed. I decided to start by caching the robots.txt file so that my bot could refer to it without sending repeated requests to the site. My bot would then check URLs against those contained in the robot.txt file and would return a "robot.txt error" rather than crawl the forbidden URL:

In [19]:
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse

def can_crawl(url):
    """
    Check if the crawler can crawl a given URL based on the site's robots.txt.
    """
    parsed_url = urlparse(url)
    robots_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"
    
    rp = RobotFileParser()
    rp.set_url(robots_url)
    rp.read()
    
    return rp.can_fetch("FriendlyUniStudentResearcher", url)

def crawl(url):
    """
    Attempt to crawl a URL, respecting robots.txt rules.
    """
    if can_crawl(url):
        try:
            response = requests.get(url)
            # Process the response here (e.g., parse HTML, follow links, etc.)
            print(f"Successfully crawled: {url}")
        except Exception as e:
            print(f"An error occurred while crawling {url}: {e}")
    else:
        print(f"robots.txt error: Crawling not allowed for {url}")

# Example usage
urls_to_crawl = [
    "http://police.uk/mediacentre",
    "http://police.uk/?u=media",
    "http://police.uk"
    # Add other URLs you're interested in
]

for url in urls_to_crawl:
    crawl(url)

robots.txt error: Crawling not allowed for http://police.uk/mediacentre
robots.txt error: Crawling not allowed for http://police.uk/?u=media
robots.txt error: Crawling not allowed for http://police.uk


It is customary to include a specific "user-agent" to identify your bot and make it possible for website administrators to contact you with concerns:

In [ ]:
session = requests.Session()

# Set the custom user-agent for all requests made with this session
session.headers.update({
    'User-Agent': "FriendlyUniStudentResearcher/1.0 (+mailto:soc204@exeter.ac.uk)"
})
response = session.get()

In [7]:
import requests
import json

# Define your custom user-agent string
user_agent = "FriendlyUniStudentResearcher/1.0 (+mailto:soc204@exeter.ac.uk)"

# Set the headers for your request to include your custom user-agent
headers = {
    'User-Agent': user_agent
}

# The URL for testing headers (httpbin.org is useful for HTTP requests testing)
test_url = "https://httpbin.org/headers"

# Make the request with your headers
response = requests.get(test_url, headers=headers)

# Parse the JSON response
response_json = response.json()

# Extract and print the User-Agent header from the response
print("User-Agent received by httpbin.org:")
print(response_json['headers']['User-Agent'])


User-Agent received by httpbin.org:
FriendlyUniStudentResearcher/1.0 (+mailto:soc204@exeter.ac.uk)


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options

options = Options()
user_agent = "FriendlyUniStudentResearcher/1.0 (+mailto:soc204@exeter.ac.uk)"
options.add_argument(f'user-agent={user_agent}')
# Specify the path to chromedriver if it's not in your PATH
chromedriver_path = r"C:\Users\socor\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Initialize the WebDriver (assuming Chrome)
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

session = requests.Session()

# Set the custom user-agent for all requests made with this session
session.headers.update({
    'User-Agent': "FriendlyUniStudentResearcher/1.0 (+mailto:soc204@exeter.ac.uk)"
})
response = session.get()

driver.quit()

In [22]:
url = 'https://www.police.uk/'
parsed_url = urlparse(url)
robots_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"
rp = RobotFileParser()
rp.set_url(robots_url)
rp.read()
for line in rp.default_entry.rulelines:
    print(f"Allow: {line.allowance} Path: {line.path}")

# Check if the root URL is allowed
print(rp.can_fetch("*", "https://www.police.uk/"))
rp.can_fetch("*", url)

AttributeError: 'NoneType' object has no attribute 'rulelines'

In [18]:
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser

# Initialize a cache dictionary
robots_cache = {}

def cache_robots_data(url):
    """
    Fetches and caches the robots.txt data for the given URL's domain.
    """
    # Parse the domain from the given URL
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    robots_url = f"{base_url}/robots.txt"
    
    # Check if we already have cached data for this domain
    if base_url in robots_cache:
        print("Using cached robots.txt data.")
        return robots_cache[base_url]
    else:
        print("Fetching new robots.txt data.")
        # Initialize a RobotFileParser instance
        rp = RobotFileParser()
        rp.set_url(robots_url)
        rp.read()  # Fetch and parse the robots.txt
        
        # Cache the RobotFileParser instance for future use
        robots_cache[base_url] = rp
        
        return rp

# Example usage
rp = cache_robots_data('https://www.police.uk')
rp

Fetching new robots.txt data.


# Data Collection

In [1]:
from selenium.webdriver.chrome.options import Options
def establish_user_agent(user_agent, chromedriver_path):
    chrome_options = Options()
    chrome_options.add_argument(f"user-agent={user_agent}")
    return chrome_options

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
def init_chrome_webdriver(chromedriver_path, chrome_options):
    chrome_options.add_argument("--no-sandbox")  # This parameter helps in avoiding unnecessary crashes.
    chrome_options.add_argument("--disable-gpu")  # Disables GPU hardware acceleration. If software renderer is not in place, then the browser might not launch.
    chrome_options.add_argument("--log-level=3")  # This will only show fatal errors in the console.
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [3]:
import time
import json
from selenium.webdriver.common.by import By 
def test_user_agent(driver, user_agent):
    driver.get("https://httpbin.org/user-agent")
    time.sleep(5)
    response_data = json.loads(driver.find_element(By.TAG_NAME, "body").text)
    echoed_user_agent = response_data["user-agent"]
    
    if echoed_user_agent != user_agent:
        print("User-Agent does not match the expected value. Quitting...")
        raise Exception("User-Agent does not match the expected value.")

In [4]:
def is_target_disallowed(target, disallowed_dict):
    """
    Check if the target path matches any of the disallowed paths.

    :param target_path: The target path to check
    :param disallowed_paths: A dictionary of disallowed paths from robots.txtf files for each base_url
    :return: True if the target path is disallowed, False otherwise
    """
    # Extract base URL from the target
    parsed_url = urlparse(target)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    
    # Retrieve the list of disallowed patterns for the base URL
    disallowed_patterns = disallowed_dict.get(base_url, [])
    
    # Normalize target path
    target_pattern = f'{parsed_url.path}?{parsed_url.query}'.rstrip("?")
    target_path = target_pattern.rstrip("/")

    for pattern in disallowed_patterns:
        # Normalize disallowed path
        pattern = pattern.rstrip("/")
        
        # Check if the target pattern starts with the disallowed pattern
        if target_path.startswith(pattern):
            return True
        # Checking for file extension disallowance, e.g., '*.aspx$'
        if pattern.endswith('$'):
            base_pattern = pattern[1:-1]
            if target_path.endswith(base_pattern):
                return True

    return False

In [5]:
from urllib.parse import urlparse
import re
def establish_bot_permissions(driver, target, existing_disallowed=None):
    parsed_url = urlparse(target)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    
    # Initialize the dictionary if not provided
    if existing_disallowed is None:
        existing_disallowed = {}
    
    # If the base URL is already in the dictionary, return it
    if base_url in existing_disallowed:
        if is_target_disallowed(target, existing_disallowed):
            print('This URL is not allowed to be crawled in line with robots.txt')
            raise Exception(f"Target path {target} is disallowed.")
        else:
            print(f"{target} is not disallowed")
        return existing_disallowed
    
    # Navigate to relevant robots.txt file
    robots_url = f"{base_url}/robots.txt"
    driver.get(robots_url)
    time.sleep(1)
    
    # Scrape disallowed patterns
    robots_txt_content = driver.find_element(By.TAG_NAME, "body").text
    disallow_pattern = r"Disallow: ([^\n]+)"
    disallowed_paths = re.findall(disallow_pattern, robots_txt_content)
    existing_disallowed[base_url] = disallowed_paths
    
    if is_target_disallowed(target, existing_disallowed):
        print('This URL is not allowed to be crawled in line with robots.txt')
        raise Exception(f"Target path {target} is disallowed.")
    else:
            print(f"{target} is not disallowed")
    return existing_disallowed

In [6]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_force_areas(driver, target):
    try:
        driver.get(target)
        all_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".js-crime-stats-table-toggle"))
        )
    
        if len(all_buttons) > 1:
            toggle_button = all_buttons[1]  # Select the second button
            driver.execute_script("arguments[0].scrollIntoView(true);", toggle_button)
            toggle_button.click()
            time.sleep(2)
        else:
            print("Not enough buttons found.")
    
        tables = driver.find_elements(By.TAG_NAME, 'table')
        table = tables[-1]
        driver.execute_script("arguments[0].scrollIntoView(true);", table)
        rows = table.find_elements(By.TAG_NAME, 'tr')
        force_areas = []
    
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if cells:
                text = cells[0].text.strip()
                force_areas.append(text)
    except Exception as e:
        print(f"An error occurred while processing: {e}")    
    return force_areas

In [7]:
from selenium.webdriver.common.keys import Keys
def navigate_to_force_area_performance(driver, area, disallowed_patterns, force_area_urls={}):
    try:
        all_search_inputs = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "input[type='search'], input[name*='search'], input[placeholder*='Search']"))
        )

        # Make sure there are at least two search bars
        if len(all_search_inputs) >= 2:
            search_input = all_search_inputs[1]  # Select the second search input
        else:
            raise Exception("Less than two search inputs found on the page.")
            
        search_input.click()
        # Clear the search field first in case there's any pre-filled text
        search_input.clear()
        # Enter the area name into the search field
        search_input.send_keys(area)
        # Search!
        search_input.send_keys(Keys.ENTER)  # Press Enter directly via Selenium
        
        time.sleep(1)
        #Check if this new page is disallowed
        target = driver.current_url
        disallowed_patterns = establish_bot_permissions(driver,target,disallowed_patterns)

        driver.get(target)
        time.sleep(1)
        print(f"Navigation to the {area} performance page is successful.")

    except Exception as e:
        print(f"An error occurred while navigating to the {area} performance page: {e}")
    return force_area_urls

In [8]:
def get_jurisdictions(driver, area, disallowed_patterns, force_area_jurisdictions={}):
    link = driver.find_elements(By.XPATH, "//a[.//h3[contains(@class, 'c-link-panel_title') and contains(text(), 'Compare your area')]]")
    if len(link)<1:
        print("No data available")
        jurisdictions[area]={}
        return jurisdictions
    link = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//a[.//h3[contains(@class, 'c-link-panel_title') and contains(text(), 'Compare your area')]]"))
        )
    target = link.get_attribute('href')
    disallowed_patterns = establish_bot_permissions(driver,target,disallowed_patterns) 
    driver.get(target)
    time.sleep(1)
    all_buttons = driver.find_elements(By.CSS_SELECTOR, ".js-crime-stats-table-toggle")
    if len(all_buttons) > 1:
        toggle_button = all_buttons[1]
        driver.execute_script("arguments[0].scrollIntoView(true);", toggle_button)
        toggle_button.click()
        time.sleep(1)
    else:
        print("Not enough buttons found.")
        jurisdictions[area]={}
        return jurisdictions
    
    tables = driver.find_elements(By.TAG_NAME, 'table')        
    table = tables[2]
    driver.execute_script("arguments[0].scrollIntoView(true);", table)
    rows = table.find_elements(By.TAG_NAME, 'tr')
    force_area_jurisdictions = {}
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if cells:
            text = cells[0].text.strip()
            force_area_jurisdictions[text]=cells[1].text.strip()
    jurisdictions[area] = force_area_jurisdictions
    return jurisdictions

In [9]:
def get_force_area_finances(driver, area, disallowed_patterns,financial_reserves={}):
    navigate_to_force_area_performance(driver, area, disallowed_patterns)
    link = driver.find_elements(By.XPATH, "//a[.//h3[contains(@class, 'c-link-panel_title') and contains(text(), 'Police financial reserves')]]")
    if len(link)<1:
        print("No data available")
        financial_reserves[area]={}
        return financial_reserves
    link = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//a[.//h3[contains(@class, 'c-link-panel_title') and contains(text(), 'Police financial reserves')]]"))
        )
    target = link.get_attribute('href')
    disallowed_patterns = establish_bot_permissions(driver,target,disallowed_patterns) 
    driver.get(target)
    time.sleep(1)
    all_buttons = driver.find_elements(By.CSS_SELECTOR, ".js-crime-stats-table-toggle")
    if len(all_buttons) > 1:
        toggle_button = all_buttons[0]
        driver.execute_script("arguments[0].scrollIntoView(true);", toggle_button)
        toggle_button.click()
        time.sleep(1)
    else:
        print("Not enough buttons found.")
        return financial_reserves
    
    tables = driver.find_elements(By.TAG_NAME, 'table')        
    table = tables[-2]
    driver.execute_script("arguments[0].scrollIntoView(true);", table)
    rows = table.find_elements(By.TAG_NAME, 'tr')
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if cells:
            year = cells[0].text.strip()
            financial_reserves[area][year]['General fund']=cells[1].text.strip()
            financial_reserves[area][year]['Earmarked reserves']=cells[2].text.strip()
            financial_reserves[area][year]['Total resource reserves']=cells[3].text.strip()
            financial_reserves[area][year]['Capital reserves']=cells[4].text.strip()
    return financial_reserves
# {force_area:{Mar 2018: {General Fund: 10000, Earmarked Reserves: 10000, Total Resource Reserves: 200000, Capital Reserves: 50000} {Mar 2017:{General Fund....}}}}
# {force_area_keys:{Year_keys:{Fund_type_keys:Values}}}

What follows is the webscraping script- remember to recreate this script's output, you must have first downloaded the relevant chromedriver for your machine from https://googlechromelabs.github.io/chrome-for-testing/#stable, and provide the path to your own version of the chromedriver where prompted in the script. You may also wish to use your own user-agent. It is recommended that your user-agent contains a (+mailto:emailaddress) string so that any crawling of the bot that raises concerns with the service provider can be mediated by them reaching out to you.

In [10]:
# Setup User-Agent
user_agent = "FriendlyUniStudentResearcher/1.0 (+mailto:soc204@exeter.ac.uk)"

#Provide the path to your own version of the chromedriver
chromedriver_path = r"C:\Users\socor\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

chrome_options = establish_user_agent(user_agent, chromedriver_path)

# Initialize the WebDriver (assuming Chrome)
driver = init_chrome_webdriver(chromedriver_path,chrome_options)

# Set target URL
target = 'https://www.police.uk/pu/your-area/avon-somerset-constabulary/performance/financial-reserves/'

try:
    # Navigate to a website that echoes back the user-agent
    test_user_agent(driver, user_agent)
        
    # Navigate to target website robots.txt and save the disallowed patterns
    disallowed_patterns = establish_bot_permissions(driver, target)
    
    # Collect the names of Force areas for which data is available
    Force_Areas = get_force_areas(driver, target)
    
    target = 'https://www.police.uk/pu/performance/'
    disallowed_patterns = establish_bot_permissions(driver, target, disallowed_patterns)
    driver.get(target)
    force_area_urls = {}
    jurisdictions = {}
    financial_reserves = {}
    Periods = ('Mar 2011', 'Mar 2012', 'Mar 2013', 'Mar 2014', 'Mar 2015', 'Mar 2016', 'Mar 2017', 'Mar 2018')
    Reserves = ('General fund','Earmarked reserves', 'Total resource reserves', 'Capital reserves')
    for area in Force_Areas:
        period_dict={}
        for period in Periods:
            reserves_dict={}
            for reserve_type in Reserves:
                reserves_dict[reserve_type] = None
                period_dict[period] = reserves_dict 
                financial_reserves[area] = period_dict
    # Target each force area's performance data
    for area in Force_Areas[:-1]:
        force_area_urls = navigate_to_force_area_performance(driver, area, disallowed_patterns, force_area_urls)
        jurisdictions = get_jurisdictions(driver, area, disallowed_patterns)
        #get force area's historical financial reserves
        financial_reserves = get_force_area_finances(driver, area, disallowed_patterns,financial_reserves)
        
        driver.get('https://www.police.uk/pu/performance/')
        time.sleep(2)    

    time.sleep(10)
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()

https://www.police.uk/pu/your-area/avon-somerset-constabulary/performance/financial-reserves/ is not disallowed
https://www.police.uk/pu/performance/ is not disallowed
https://www.police.uk/pu/your-area/avon-somerset-constabulary/performance/performance-avon-somerset/?tc=AN004 is not disallowed
Navigation to the Avon and Somerset Constabulary performance page is successful.
https://www.police.uk/pu/your-area/avon-somerset-constabulary/performance/compare-your-area/?tc=AN004 is not disallowed
https://www.police.uk/pu/your-area/avon-somerset-constabulary/performance/performance-avon-somerset/?tc=AN004 is not disallowed
Navigation to the Avon and Somerset Constabulary performance page is successful.
https://www.police.uk/pu/your-area/avon-somerset-constabulary/performance/financial-reserves/?tc=AN004 is not disallowed
https://www.police.uk/pu/your-area/bedfordshire-police/performance/performance-bedfordshire-police/?tc=NU1 is not disallowed
Navigation to the Bedfordshire Police performanc

In [18]:
print(financial_reserves)

{'Avon and Somerset Constabulary': {'Mar 2011': {'General fund': '£6.7m', 'Earmarked reserves': '£25.6m', 'Total resource reserves': '£32.2m', 'Capital reserves': '£2.1m'}, 'Mar 2012': {'General fund': '£7.5m', 'Earmarked reserves': '£29.6m', 'Total resource reserves': '£37.1m', 'Capital reserves': '£2.4m'}, 'Mar 2013': {'General fund': '£13.6m', 'Earmarked reserves': '£28.7m', 'Total resource reserves': '£42.2m', 'Capital reserves': '£1.9m'}, 'Mar 2014': {'General fund': '£14.4m', 'Earmarked reserves': '£31.3m', 'Total resource reserves': '£45.7m', 'Capital reserves': '£4.1m'}, 'Mar 2015': {'General fund': '£14.4m', 'Earmarked reserves': '£28.8m', 'Total resource reserves': '£43.2m', 'Capital reserves': '£17.0m'}, 'Mar 2016': {'General fund': '£10.4m', 'Earmarked reserves': '£29.3m', 'Total resource reserves': '£39.7m', 'Capital reserves': '£11.6m'}, 'Mar 2017': {'General fund': '£10.4m', 'Earmarked reserves': '£26.7m', 'Total resource reserves': '£37.1m', 'Capital reserves': '£7.1m'}

# Data Cleaning

The webpage states that data is not available for "City of London Police" force area, so we'll add that force area manually. Similarly, most of the desired data was unavailable for the aggregate "Total England & Wales", so we need to add empty dictionaries to represent missing values in our data for those two "force areas".

In [20]:
Force_Areas.append("City of London Police")

We need to store that data in a pandas series to unlock better functionality. The idea is to get force area level data on financial reserves over the period since records begin and average crime rate for each force area last year.

In [97]:
import pandas as pd
import numpy as np
Force_Area = pd.Index(Force_Areas)
force_avg_crime_rate = {}
for area in Force_Areas:
    if area in jurisdictions:
        if jurisdictions[area] == {}:
            force_avg_crime_rate[area]=np.nan
            continue
        force_avg_crime_rate[area]=jurisdictions[area]['Force average']
    else:
        force_avg_crime_rate[area]=np.nan
Force_Crime_Rates = pd.Series(force_avg_crime_rate,name='Average Crime Rate')
force_jurisdictions = {}
for area in Force_Areas:
    if area in jurisdictions:
        area_jurisdictions=[]
        for jurisdiction in jurisdictions[area]:
            if jurisdiction == 'Force average':
                continue
            area_jurisdictions.append(jurisdiction)
        force_jurisdictions[area]=(area_jurisdictions) 
    else:
        force_jurisdictions[area]=np.nan
Force_Jurisdictions = pd.Series(force_jurisdictions, name='Jurisdictions')
ForceAreas = pd.concat([Force_Crime_Rates,Force_Jurisdictions], axis=1)
FA_series1 = {}
FA_series2 = {}
inner_keys=[]
for area in Force_Areas:
    for period in Periods:
        for fund in Reserves:
            inner_key = f"{period} {fund}"
            if area not in financial_reserves:
                FA_series1[area]={inner_key:np.nan}
                continue
            if period not in financial_reserves[area]:
                FA_series1[area]={inner_key:np.nan}
                continue
            FA_series1[area]={inner_key:financial_reserves[area][period][fund]}
            for ik in FA_series1[area]:
                if area not in FA_series1:
                    continue
                inner_keys.append(ik)
unique_iks = set(inner_keys)
area_fund={}
#for key in unique_iks:
#    for area in Force_Areas:
#        if key not in FA_series1[area]:
#            continue
#        area_fund[area]=FA_series1[area][key]
#    #print(area_fund)
#    series = pd.Series(area_fund,name=key)
#    #print(series)
#    ForceAreas=pd.concat([ForceAreas,series],axis=1)
FA_series2 = FA_series1
for key in unique_iks:
    for area in Force_Areas:
        for period in Periods:
            for fund in Reserves:
                inner_key = f"{period} {fund}"
                if area not in financial_reserves:
                    FA_series2[area]={inner_key:np.nan}
                    continue
                if period not in financial_reserves[area]:
                    FA_series2[area]={inner_key:np.nan}
                    continue
                FA_series2[area]={inner_key:financial_reserves[area][period][fund]}
                print(FA_series2)
                for ik in FA_series2[area]:
                    if area not in FA_series2:
                        continue
                    if key not in FA_series2[area]:
                        continue
            area_fund[area]=FA_series2[area][key]
    #print(area_fund)
    series = pd.Series(area_fund,name=key)
    #print(series)
    ForceAreas=pd.concat([ForceAreas,series],axis=1)

ForceAreas
#FA_series2[area]=FA_series1[area][inner_key]
#print(FA_series2)

{'Avon and Somerset Constabulary': {'Mar 2011 General fund': '£6.7m'}, 'Bedfordshire Police': {'Mar 2018 Capital reserves': '£0.0m'}, 'Cambridgeshire Constabulary': {'Mar 2018 Capital reserves': '£1.9m'}, 'Cheshire Constabulary': {'Mar 2018 Capital reserves': '£5.1m'}, 'Cleveland Police': {'Mar 2018 Capital reserves': '£1.0m'}, 'Cumbria Constabulary': {'Mar 2018 Capital reserves': '£7.2m'}, 'Derbyshire Constabulary': {'Mar 2018 Capital reserves': '£4.8m'}, 'Devon & Cornwall Police': {'Mar 2018 Capital reserves': '£3.8m'}, 'Dorset Police': {'Mar 2018 Capital reserves': '£1.2m'}, 'Durham Constabulary': {'Mar 2018 Capital reserves': '£9.4m'}, 'Dyfed-Powys Police': {'Mar 2018 Capital reserves': '£0.8m'}, 'Essex Police': {'Mar 2018 Capital reserves': '£1.9m'}, 'Gloucestershire Constabulary': {'Mar 2018 Capital reserves': '£3.5m'}, 'Greater Manchester Police': {'Mar 2018 Capital reserves': '£26.7m'}, 'Gwent Police': {'Mar 2018 Capital reserves': '£1.7m'}, 'Hampshire Constabulary': {'Mar 2018

KeyError: 'Mar 2018 Capital reserves'

In [99]:
import pandas as pd
import numpy as np

# Assuming 'Force_Areas', 'jurisdictions', 'Periods', 'Reserves', 'financial_reserves' are defined
force_avg_crime_rate = {}
force_jurisdictions = {}
financial_data = {}

# Process average crime rates and jurisdictions
for area in Force_Areas:
    force_avg_crime_rate[area] = jurisdictions.get(area, {}).get('Force average', np.nan)
    force_jurisdictions[area] = [j for j in jurisdictions.get(area, {}) if j != 'Force average'] if area in jurisdictions else np.nan

# Create Pandas Series
Force_Crime_Rates = pd.Series(force_avg_crime_rate, name='Average Crime Rate')
Force_Jurisdictions = pd.Series(force_jurisdictions, name='Jurisdictions')

# Create DataFrame from Series
ForceAreas = pd.DataFrame({'Average Crime Rate': Force_Crime_Rates, 'Jurisdictions': Force_Jurisdictions})

# Process financial reserves
for area in Force_Areas:
    area_data = financial_reserves.get(area, {})
    for period in Periods:
        for fund in Reserves:
            key = f"{period} {fund}"
            value = area_data.get(period, {}).get(fund, np.nan)
            financial_data.setdefault(key, {})[area] = value

# Add financial data to DataFrame
for key, values in financial_data.items():
    ForceAreas[key] = pd.Series(values)

ForceAreas

,Average Crime Rate,Jurisdictions,Mar 2011 General fund,Mar 2011 Earmarked reserves,Mar 2011 Total resource reserves,Mar 2011 Capital reserves,Mar 2012 General fund,Mar 2012 Earmarked reserves,Mar 2012 Total resource reserves,Mar 2012 Capital reserves,...,Mar 2016 Total resource reserves,Mar 2016 Capital reserves,Mar 2017 General fund,Mar 2017 Earmarked reserves,Mar 2017 Total resource reserves,Mar 2017 Capital reserves,Mar 2018 General fund,Mar 2018 Earmarked reserves,Mar 2018 Total resource reserves,Mar 2018 Capital reserves
Avon and Somerset Constabulary,83.24,"[Bath & North East Somerset, South Gloucesters...",£6.7m,£25.6m,£32.2m,£2.1m,£7.5m,£29.6m,£37.1m,£2.4m,...,£39.7m,£11.6m,£10.4m,£26.7m,£37.1m,£7.1m,£10.4m,£25.3m,£35.7m,£0.3m
Bedfordshire Police,73.8,"[Central Bedfordshire, Bedford, Luton]",£2.9m,£6.5m,£9.4m,£0.4m,£2.9m,£6.6m,£9.5m,£0.4m,...,£13.7m,£0.0m,£3.0m,£10.2m,£13.2m,£0.6m,£3.0m,£6.5m,£9.5m,£0.0m
Cambridgeshire Constabulary,84.51,"[East Cambridgeshire, South Cambridgeshire, Hu...",£4.8m,£13.3m,£18.1m,£18.2m,£7.0m,£19.1m,£26.1m,£26.3m,...,£26.6m,£1.7m,£7.2m,£21.3m,£28.5m,£1.2m,£7.2m,£15.3m,£22.5m,£1.9m
Cheshire Constabulary,78.23,"[Cheshire East, Cheshire West, Warrington, Hal...",£7.8m,£10.3m,£18.2m,£2.4m,£5.2m,£10.4m,£15.6m,£4.4m,...,£20.9m,£7.1m,£5.8m,£8.5m,£14.3m,£5.8m,£6.3m,£5.0m,£11.3m,£5.1m
Cleveland Police,144.57,"[Stockton-on-Tees, Redcar & Cleveland, Hartlep...",£7.1m,£6.0m,£13.1m,£0.3m,£8.2m,£6.0m,£14.2m,£0.3m,...,£15.2m,£0.4m,£8.0m,£6.4m,£14.4m,£0.7m,£6.1m,£7.2m,£13.3m,£1.0m
Cumbria Constabulary,74.1,"[South Lakeland, Eden, Copeland, Allerdale, Ba...",£5.1m,£16.8m,£22.0m,£3.3m,£5.1m,£18.9m,£24.1m,£3.3m,...,£23.9m,£6.0m,£3.0m,£21.3m,£24.3m,£6.3m,£3.0m,£19.9m,£22.9m,£7.2m
Derbyshire Constabulary,85.62,"[Derbyshire Dales, North East Derbyshire, Sout...",£3.3m,£31.7m,£35.0m,£6.2m,£3.3m,£36.9m,£40.2m,£6.5m,...,£39.5m,£9.3m,£4.0m,£28.2m,£32.2m,£4.8m,£4.0m,£20.8m,£24.8m,£4.8m
Devon & Cornwall Police,58.6,"[Isles of Scilly, South Devon & Dartmoor, East...",£6.2m,£22.3m,£28.5m,£6.2m,£6.2m,£32.1m,£38.3m,£6.6m,...,£63.7m,£4.7m,£11.9m,£46.9m,£58.8m,£6.1m,£11.7m,£38.4m,£50.0m,£3.8m
Dorset Police,66.92,"[Dorset County, Poole, Bournemouth]",£3.3m,£14.4m,£17.7m,£0.0m,£4.0m,£14.0m,£18.0m,£0.3m,...,£13.0m,£3.9m,£7.4m,£5.7m,£13.1m,£3.9m,£5.0m,£3.8m,£8.8m,£1.2m
Durham Constabulary,106.92,"[County Durham, Darlington]",£6.1m,£9.5m,£15.6m,£0.0m,£6.1m,£11.3m,£17.3m,£0.0m,...,£12.7m,£0.8m,£5.7m,£7.3m,£13.0m,£5.3m,£5.7m,£3.1m,£8.7m,£9.4m


In [29]:
print(jurisdictions)
#Jurisdictions=pd.DataFrame(Index = Force_Area, jurisdictions)
#Jurisdictions

{'Avon and Somerset Constabulary': {'Force average': '83.24', 'Bath & North East Somerset': '64.78', 'South Gloucestershire': '65.18', 'Somerset': '70.55', 'North Somerset': '71.27', 'Bristol': '118.47'}, 'Bedfordshire Police': {'Force average': '73.8', 'Central Bedfordshire': '55.34', 'Bedford': '78.27', 'Luton': '88.51'}, 'Cambridgeshire Constabulary': {'Force average': '84.51', 'East Cambridgeshire': '48.32', 'South Cambridgeshire': '49.05', 'Huntingdonshire': '61.03', 'Fenland': '79.24', 'Peterborough': '120.79', 'Cambridge': '121.05'}, 'Cheshire Constabulary': {'Force average': '78.23', 'Cheshire East': '69.88', 'Cheshire West': '77.20', 'Warrington': '78.48', 'Halton': '102.31'}, 'Cleveland Police': {'Force average': '144.57', 'Stockton-on-Tees': '125.09', 'Redcar & Cleveland': '125.97', 'Hartlepool': '153.47', 'Middlesbrough': '183.78'}, 'Cumbria Constabulary': {'Force average': '74.1', 'South Lakeland': '52.42', 'Eden': '60.74', 'Copeland': '66.37', 'Allerdale': '72.45', 'Barro

We also want a dataframe that combines jurisdictions with their crime rates (excluding force area averages) so that we can see the worst 5 jurisdictions for crime rate and top 5 jurisdictions for crime rate. From that we need a dataframe linking each jurisdiction a force area so that we can identify which force areas have the most success and whether reserve resources are an effective predictor of reduced crime rate. 

# Data Analysis

https://github.com/SOCStudentUoE/BEE2041-Empirical-Assignment